# DATA 620 - Assignment 3 Alt

by: Jeremy OBrien and Mael Illien

Load a graph database of your choosing from a text file or other source. If you take a large network dataset from the web (such as from Stanford Large Network Dataset Collection (https://snap.stanford.edu/data/)), please feel free at this point to load just a small subset of the nodes and edges.

Create basic analysis on the graph, including the graph’s diameter, and at least one other metric of your choosing. You may either code the functions by hand (to build your intuition and insight), or use functions in an existing package.

Use a visualization tool of your choice (Neo4j, Gephi, etc.) to display information. Please record a short video (~ 5 minutes), and submit a link to the video in advance of our meet-up.

In [113]:
import os
import random
import pandas as pd
import networkx as nx

Data sourced from: https://snap.stanford.edu/data/soc-sign-bitcoin-otc.html

In [104]:
# Import data
url = 'https://raw.githubusercontent.com/JeremyOBrien16/CUNY_DATA_620/master/Assignment03/soc-sign-bitcoinotc.csv'
ratings = pd.read_csv(url, names=['SOURCE','TARGET','RATING','TIMESTAMP'] )
ratings

,SOURCE,TARGET,RATING,TIMESTAMP
0,6,2,4,1.289242e+09
1,6,5,2,1.289242e+09
2,1,15,1,1.289243e+09
3,4,3,7,1.289245e+09
4,13,16,8,1.289254e+09
...,...,...,...,...
35587,4499,1810,1,1.453612e+09
35588,2731,3901,5,1.453679e+09
35589,2731,4897,5,1.453679e+09
35590,13,1128,1,1.453680e+09


In [36]:
data = open('soc-sign-bitcoinotc.csv', 'r')
graphtype = nx.DiGraph()
graph = nx.parse_edgelist(data, 
                      delimiter=',', 
                      create_using=graphtype,
                      nodetype=int, 
                      data=(('weight', float),('time', float)))

In [37]:
graph

In [141]:
len(graph.nodes)

5881

In [140]:
len(graph.edges)

35592

In [173]:
# Query format for node creation
"CREATE (:User {id: '6', target: '5'})"

"CREATE (:User {id: '6', target: '5'})"

In [180]:
ratings[ratings['SOURCE'].isin(random.sample(users, 100))]

,SOURCE,TARGET,RATING,TIMESTAMP
190,88,7,3,1.294974e+09
356,93,81,2,1.297844e+09
462,177,161,1,1.298778e+09
777,88,60,4,1.302280e+09
778,88,104,3,1.302280e+09
...,...,...,...,...
34762,5852,5906,1,1.424049e+09
34763,5852,5909,1,1.424049e+09
34764,5852,5910,1,1.424049e+09
34765,5852,2851,1,1.424049e+09


In [186]:
# Randomly sample 100 users
users = set(ratings['SOURCE']) # Identify unique users

#user_subset = ratings[ratings['SOURCE'].isin(random.sample(users, 100))]
user_subset = random.sample(users,100)
                      
# Delete the file if it already exists
if os.path.exists("create_nodes.txt"):
      os.remove("create_nodes.txt")

# Write the queries to file
with open("create_nodes.txt", "a") as f:     
   
    for node in user_subset:
        query = "CREATE (:User {id: '%d'}) \n" % (node)
        f.write(query)

In [188]:
# Query format for relationship creation
#CREATE (:User {id: '112'})-[:TRUSTS {rating: 2}]->(:User {id: '64'})

"MATCH (u1:User {id: '6'}),(u2:User {id: '2'}) CREATE (u1)-[:TRUSTS {rating: 4}]->(u2) "

"MATCH (u1:User {id: '6'}),(u2:User {id: '2'}) CREATE (u1)-[:TRUSTS {rating: 4}]->(u2) "

In [189]:
edges_subset = ratings[ratings['SOURCE'].isin(user_subset)]
edges_subset.index

Int64Index([   25,    26,    27,   210,   221,   353,   370,   372,   427,
              483,
            ...
            35165, 35166, 35204, 35248, 35350, 35365, 35426, 35507, 35523,
            35541],
           dtype='int64', length=522)

In [190]:
# Obtain the dataframe rows containing relationships of the user subset
edges_subset = ratings[ratings['SOURCE'].isin(user_subset)]

# Delete the file if it already exists
import os
if os.path.exists("create_relationships.txt"):
      os.remove("create_relationships.txt")

# Write the queries to file
with open("create_relationships.txt", "a") as f: 
    for i in edges_subset.index:
        src = edges_subset['SOURCE'][i]
        tar = edges_subset['TARGET'][i]
        rating = edges_subset['RATING'][i]
        query1 = "MATCH (u1:User {id: '%s'}),(u2:User {id: '%s'}) WITH u1,u2 CREATE (u1)-[:TRUSTS {rating: %s}]->(u2) \n" % (src, tar, rating) 
        query2 = "CREATE (:User {id: '%s'})-[:TRUSTS {rating: %s}]->(:User {id: '%s'}) \n" % (src, tar, rating)
        query3 = "MATCH (u1:User),(u2:User) WHERE u1.id = '%s' AND u2.id = '%s' CREATE (u1)-[:TRUSTS {rating: %s}]->(u2) \n"  % (src, tar, rating)
        f.write(query3)
        

In [149]:
# Easy load csv query from github?
#LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/JeremyOBrien16/CUNY_DATA_620/master/Assignment03/soc-sign-bitcoinotc.csv" AS row
#CREATE (n:Category)
#SET n = row

SyntaxError: invalid syntax (<ipython-input-149-71f2640acb6e>, line 2)

In [ ]:
MATCH (u1:User {id: '1'}) WITH u1 MATCH (u1:User {id: '5'}) CREATE (u1)-[:TRUSTS]->(u2)

In [ ]:
MATCH (a:Person),(b:Person)
WHERE a.name = 'A' AND b.name = 'B'
CREATE (a)-[r:RELTYPE]->(b)
RETURN type(r)

In [ ]:
MATCH (u1:User)
WITH u1
MATCH (u2:User) 
WHERE g.chromosomeID = c.chromosomeID
CREATE (g)-[:PART_OF]->(c);